In [48]:
import pandas as pd
from tqdm import tqdm

In [49]:
clv = True
NUM_THREAD = 15
STATUS = {0:"commit", 2:"abort"}
def get_thd(txn):
    return int(txn) % NUM_THREAD
def get_lock(lock_idx):
    lock_idx = int(lock_idx)
    if lock_idx == 0:
        return "SH"
    elif lock_idx == 1:
        return "EX"
    else:
        return "NONE"
def rm_txn(row, place, txn):
    thread = get_thd(txn)
    members = locktable[row][place]
    found = False
    for i, step in enumerate(members):
        if step[0] == txn:
            found = True
            break
    if not found:
        print("not found: {}".format(line))
        raise
    locktable[row][place] = members[:i] + members[i+1:]
    return thread, members[i]
def get_schedule(start, end, thread):
    return schedule.loc[start:end+1, thread][schedule.loc[start:end+1, thread].notnull()]
def find_locktable_with_txn(txn):
    for row in locktable:
        if txn in [item[0] for place in ['owners', 'waiters', 'retired'] for item in locktable[row][place] ]:
            print(row)
            print(locktable[row])
def find_locktable_with_row(row):
    print(locktable[row])
def get_p(txn):
    if txn in priority:
        p = priority[txn]
    else:
        p = -1
    return p

In [78]:
def parse(line):
    txn = ""
    tokens = line.split(" ")
    if "[thread]" in line:
        if "start running txn" in line:
            thread = int(tokens[2])
            txn = tokens[6]
            schedule.loc[ts, thread] = "start txn %s"%txn
            if clv:
                barriers[txn] = 0
        elif "finish txn" in line:
            txn = tokens[3]
            thread = get_thd(txn)
            rc = tokens[-1]
            # "[thread-%lu] finish txn %lu (aborted)\n"
            schedule.loc[ts, thread] = "thread finish txn %s %s"%(txn, thread)
            
    elif "[row_" in tokens[0]:
            #return
            if "add txn" in line:
                    #[row_ww] add txn 1 type 1 to owners of row 1449388428864366
                    txn = tokens[3]
                    lock = get_lock(tokens[5])
                    row = tokens[-1]
                    thread = get_thd(txn)
                    if row not in locktable:
                        locktable[row] = {}
                        locktable[row]['owners'] = []
                        locktable[row]['waiters'] = []
                        if clv:
                            locktable[row]['retired'] = []
                    
                    locktable[row][tokens[7]].append((txn, get_p(txn), lock))
                    if "owner" in tokens[7]:
                        schedule.loc[ts, thread] = "acquire %s lock %s"%(lock, row)
                    elif "waiter" in tokens[7]:
                        schedule.loc[ts, thread] = "wait %s lock %s"%(lock, row)
                    elif "retired" in tokens[7]:
                        schedule.loc[ts, thread] = "retire %s lock %s"%(lock, row)
            elif "rm another txn" in line:
                    # [row_clv] txn %lu rm another txn %lu from owners of row %lu
                    wounder = tokens[2]
                    txn = tokens[6]
                    row = tokens[-1]
                    place = tokens[8]
                    thread, _ = rm_txn(row, place, txn)
                    if "owner" in place:
                        schedule.loc[ts, thread] = "removed lock %s from owners by %s"%(row, wounder)
                    elif "retired" in place:
                        schedule.loc[ts, thread] = "removed lock %s from retired by %s"%(row, wounder)
            elif "rm txn" in line:
                    txn = tokens[3]
                    row = tokens[-1]
                    thread, _ = rm_txn(row, tokens[5], txn)
                    if "owner" in tokens[5]:
                        schedule.loc[ts, thread] = "remove lock %s from owners"%row
                    elif "waiter" in tokens[5]:
                        schedule.loc[ts, thread] = "remove lock %s from waiters"%row
                    elif "retired" in tokens[5]:
                        schedule.loc[ts, thread] = "release lock %s from retired"%row
            elif "from waiter to owner" in line or "from waiters to owners" in line:
                    # [row_ww] bring 0 from waiter to owner of row 1454103086539417
                    txn = tokens[2]
                    row = tokens[-1]
                    thread = get_thd(txn)
                    found = False
                    thread, entry = rm_txn(row, "waiters", txn)
                    entry = (entry[0], get_p(txn), entry[-1])
                    locktable[row]["owners"].append(entry)
                    schedule.loc[ts, thread] = "acquire lock %s"%row
            elif "from owners to retired" in line:
                    # [row_clv] move txn 0 from owners to retired type 0 of row 1459774078579136
                    txn = tokens[3]
                    row = tokens[-1]
                    thread = get_thd(txn)
                    found = False
                    lock = get_lock(tokens[-4])
                    locktable[row]["retired"].append((txn, get_p(txn), lock))
                    schedule.loc[ts, thread] = "retire lock %s"%row
            elif "abort txn" in line:
                # [row_ww] txn 8 abort txn 6 on row 10
                wound = tokens[2]
                abort = tokens[-4]
                row = tokens[-1]
                txn = wound
                schedule.loc[ts, get_thd(wound)] = "wound %s on %s"%(abort, row)
                schedule.loc[ts, get_thd(abort)] = "%s wounded by %s"%(abort,wound)
    elif "[txn]" in tokens[0]:
        if "set txn" in line and "to abort" in line:
            wounder = tokens[1]
            txn = tokens[4]
            schedule.loc[ts, get_thd(txn)] = "txn %s set abort by %s"% (txn, wounder)
            schedule.loc[ts, get_thd(wounder)] = "wound txn %s"%txn
        elif "set ts" in line:
            #[txn] set ts %lu for txn %lu\n
            txn = tokens[-1]
            p = tokens[3]
            if txn in priority:
                print("change priority of txn %s from %s to %s"%(txn, priority[txn], p))
            else:
                print("set txn %s ts %s"%(txn, p))
            priority[txn] = p
            schedule.loc[ts, get_thd(txn)] = "set ts %s"%p
        elif "is set to" in line:
            # [txn] txn %lu is set to commited
            txn = tokens[2]
            status = tokens[-1]
            thread = get_thd(txn)
            schedule.loc[ts, thread] = "txn %s set %s"%(txn, status) 
            print("txn %s set %s"%(txn, status) )
            if status != "commited":
                barriers[txn] = -1
        elif "barrier" in line:
            # "[txn] increment barrier for txn %lu\n"
            txn = tokens[-1]
            thread = get_thd(txn)
            if tokens[1] == "increment":
                barriers[txn] += 1
            elif tokens[1] == "decrement":
                barriers[txn] -= 1
            schedule.loc[ts, thread] = "txn %s %s barrier, now should be %d"%(txn, tokens[1], barriers[txn])
        elif "# bariers" in line:
            # [txn] # bariers of txn 1 = 0
            txn = tokens[-3]
            thread = get_thd(txn)
            schedule.loc[ts, thread] = "pre-commit: txn %s's barriers = %d"%(txn, int(tokens[-1]))
            assert(int(tokens[-1]) == barriers[txn])
    if "barrier for txn 12" in line or "1054408" in line :
        print("[{}]: {}".format(ts, line))
        print(locktable['547288671054408'])

In [79]:
output = open('debug.out')

In [80]:
locktable = {} # row_id: { owners: {}, waiters: {}, (opt) retired: {}}
barriers = {}
priority = {}
schedule = pd.DataFrame()
ts = 0

In [82]:
for line in tqdm(output):
    try:
        parse(line.strip())

        ts += 1
    except:
        print(line)
        raise

246it [00:00, 1198.90it/s]

[18]: [row_clv] add txn 14 type 0 to waiters of row 547288671054408
{'owners': [], 'waiters': [('14', -1, 'SH')], 'retired': []}
[19]: [row_clv] bring 14 from waiters to owners of row 547288671054408
{'owners': [('14', -1, 'SH')], 'waiters': [], 'retired': []}
[20]: [row_clv] rm txn 14 from owners of row 547288671054408
{'owners': [], 'waiters': [], 'retired': []}
[24]: [row_clv] move txn 14 from owners to retired type 0 of row 547288671054408
{'owners': [], 'waiters': [], 'retired': [('14', -1, 'SH')]}
set txn 14 ts 1
set txn 6 ts 2
[40]: [row_clv] add txn 6 type 1 to waiters of row 547288671054408
{'owners': [], 'waiters': [('6', '2', 'EX')], 'retired': [('14', -1, 'SH')]}
[44]: [row_clv] bring 6 from waiters to owners of row 547288671054408
{'owners': [('6', '2', 'EX')], 'waiters': [], 'retired': [('14', -1, 'SH')]}
[47]: [row_clv] rm txn 6 from owners of row 547288671054408
{'owners': [], 'waiters': [], 'retired': [('14', -1, 'SH')]}
[48]: [row_clv] move txn 6 from owners to retire

463it [00:00, 1136.65it/s]

change priority of txn 14 from 1 to 6
set txn 13 ts 7
[331]: [row_clv] add txn 13 type 0 to waiters of row 547288671054408
{'owners': [('12', '4', 'EX')], 'waiters': [('13', '7', 'SH')], 'retired': [('14', -1, 'SH'), ('6', '2', 'EX'), ('1', '3', 'SH'), ('2', '5', 'EX')]}
set txn 4 ts 8
[333]: [row_clv] add txn 4 type 1 to waiters of row 547288671054408
{'owners': [('12', '4', 'EX')], 'waiters': [('13', '7', 'SH'), ('4', '8', 'EX')], 'retired': [('14', -1, 'SH'), ('6', '2', 'EX'), ('1', '3', 'SH'), ('2', '5', 'EX')]}
set txn 3 ts 9
[335]: [row_clv] add txn 3 type 0 to waiters of row 547288671054408
{'owners': [('12', '4', 'EX')], 'waiters': [('13', '7', 'SH'), ('4', '8', 'EX'), ('3', '9', 'SH')], 'retired': [('14', -1, 'SH'), ('6', '2', 'EX'), ('1', '3', 'SH'), ('2', '5', 'EX')]}
set txn 10 ts 10
[337]: [row_clv] add txn 10 type 1 to waiters of row 547288671054408
{'owners': [('12', '4', 'EX')], 'waiters': [('13', '7', 'SH'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', 'EX')], 'ret

656it [00:00, 1026.03it/s]

...

txn 2 set commited
[916]: [row] try to release lock 547288671054408 for txn 2
{'owners': [('9', '11', 'EX'), ('8', '12', 'EX')], 'waiters': [('11', '13', 'SH'), ('0', '14', 'SH'), ('5', '15', 'EX'), ('7', '16', 'EX'), ('28', -1, 'SH'), ('21', '18', 'SH'), ('16', '19', 'SH')], 'retired': [('2', '5', 'EX'), ('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', 'EX')]}
[917]: [row_clv] rm txn 2 from retired of row 547288671054408
{'owners': [('9', '11', 'EX'), ('8', '12', 'EX')], 'waiters': [('11', '13', 'SH'), ('0', '14', 'SH'), ('5', '15', 'EX'), ('7', '16', 'EX'), ('28', -1, 'SH'), ('21', '18', 'SH'), ('16', '19', 'SH')], 'retired': [('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', 'EX')]}
set txn 17 ts 20
[922]: [row_clv] add txn 17 type 1 to waiters of row 547288671054408
{'owners': [('9', '11', 'EX'), ('8', '12', 'EX')], 'waiters': [('11', '13', 'SH'), ('0', '14', 'SH'), ('5', '15', 'EX'), ('7', '16', 'EX'), ('28',

1234it [00:01, 961.20it/s] 

[1090]: [row_clv] rm txn 5 from owners of row 547288671054408
{'owners': [('7', '16', 'EX')], 'waiters': [('28', -1, 'SH'), ('21', '18', 'SH'), ('16', '19', 'SH'), ('17', '20', 'EX')], 'retired': [('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', 'EX'), ('9', '11', 'EX'), ('8', '12', 'EX'), ('11', '13', 'SH'), ('0', '14', 'SH')]}
[1091]: [row_clv] move txn 5 from owners to retired type 1 of row 547288671054408
{'owners': [('7', '16', 'EX')], 'waiters': [('28', -1, 'SH'), ('21', '18', 'SH'), ('16', '19', 'SH'), ('17', '20', 'EX')], 'retired': [('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', 'EX'), ('9', '11', 'EX'), ('8', '12', 'EX'), ('11', '13', 'SH'), ('0', '14', 'SH'), ('5', '15', 'EX')]}
[1191]: [row_clv] rm txn 7 from owners of row 547288671054408
{'owners': [], 'waiters': [('28', -1, 'SH'), ('21', '18', 'SH'), ('16', '19', 'SH'), ('17', '20', 'EX')], 'retired': [('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8

1378it [00:01, 1021.95it/s]

[1304]: [row_clv] rm txn 16 from owners of row 547288671054408
{'owners': [], 'waiters': [('17', '20', 'EX')], 'retired': [('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', 'EX'), ('9', '11', 'EX'), ('8', '12', 'EX'), ('11', '13', 'SH'), ('0', '14', 'SH'), ('5', '15', 'EX'), ('7', '16', 'EX'), ('28', -1, 'SH'), ('21', '18', 'SH')]}
[1305]: [row_clv] move txn 16 from owners to retired type 0 of row 547288671054408
{'owners': [], 'waiters': [('17', '20', 'EX')], 'retired': [('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', 'EX'), ('9', '11', 'EX'), ('8', '12', 'EX'), ('11', '13', 'SH'), ('0', '14', 'SH'), ('5', '15', 'EX'), ('7', '16', 'EX'), ('28', -1, 'SH'), ('21', '18', 'SH'), ('16', '19', 'SH')]}
[1307]: [row_clv] bring 17 from waiters to owners of row 547288671054408
{'owners': [('17', '20', 'EX')], 'waiters': [], 'retired': [('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', '

In [54]:
import numpy as np
for i in schedule:
    print("thread {}: {}".format(i, schedule[i][schedule[i].notnull()].values[-1]))

thread 14: txn 29 decrement barrier, now should be 1
thread 2: retire lock 547288830341325
thread 0: pre-commit: txn 0's barriers = 2
thread 6: pre-commit: txn 21's barriers = 2
thread 3: pre-commit: txn 3's barriers = 1
thread 10: pre-commit: txn 10's barriers = 3
thread 9: pre-commit: txn 9's barriers = 1
thread 1: pre-commit: txn 16's barriers = 2
thread 13: pre-commit: txn 28's barriers = 1
thread 4: pre-commit: txn 4's barriers = 1
thread 12: pre-commit: txn 12's barriers = 1
thread 8: pre-commit: txn 8's barriers = 2
thread 11: pre-commit: txn 11's barriers = 2
thread 5: pre-commit: txn 5's barriers = 2
thread 7: pre-commit: txn 7's barriers = 2


In [61]:
find_locktable_with_txn('12')

547288671054408
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'EX'), ('29', -1, 'EX'), ('4', '8', 'EX'), ('3', '9', 'SH'), ('10', '10', 'EX'), ('9', '11', 'EX'), ('8', '12', 'EX'), ('11', '13', 'SH'), ('0', '14', 'SH'), ('5', '15', 'EX'), ('7', '16', 'EX'), ('28', -1, 'SH'), ('21', '18', 'SH'), ('16', '19', 'SH'), ('17', '20', 'EX')]}
547289226325566
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'EX')]}
547288671064569
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'SH'), ('5', '15', 'SH')]}
547288665579921
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'EX')]}
547289052598536
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'SH')]}
547288659157581
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'EX')]}
547289028221090
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'SH')]}
547288656997888
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'EX')]}
547289019994082
{'owners': [], 'waiters': [], 'retired': [('12', '4', 'SH')]}
547288659086

In [56]:
# for row in get_schedule(10454, 10619, [0,1,2]).iterrows():
#     v = " | ".join([str(x) for x in row[1].values])
#     if "115690" in v or "160397" in v:
#         print(v)

In [57]:
#schedule.to_csv('sample_schedule.csv', index=False)

In [58]:
for b in barriers:
    if barriers[b] > 0:
        print("{} = {} at thread {}".format(b, barriers[b], int(b)%NUM_THREAD))

0 = 2 at thread 0
3 = 1 at thread 3
10 = 3 at thread 10
9 = 1 at thread 9
4 = 1 at thread 4
12 = 1 at thread 12
8 = 2 at thread 8
11 = 2 at thread 11
5 = 2 at thread 5
7 = 2 at thread 7
29 = 1 at thread 14
28 = 1 at thread 13
21 = 2 at thread 6
16 = 2 at thread 1
17 = 2 at thread 2


In [31]:
get_schedule(1,1000, 9)

61                                   start txn 9
175                 wait EX lock 545013503296581
176                 acquire lock 545013503296581
177      remove lock 545013503296581 from owners
178                  retire lock 545013503296581
                         ...                    
312    release lock 545014001632375 from retired
314    release lock 545013503296581 from retired
317                                 start txn 24
318                                    set ts 14
319                 wait SH lock 545013503296581
Name: 9, Length: 121, dtype: object